# ECS759P Artificial Intelligence:  Coursework 2

## Question 2: Lost in the closet

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torch import optim

In [ ]:
# Load the train and test data
train_set = torchvision.datasets.FashionMNIST(root = ".", train = True,download = True, transform = transforms.ToTensor())
test_set = torchvision.datasets.FashionMNIST(root = ".", train = False,download = True, transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_set , batch_size = 32,shuffle = False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 32,shuffle = False)

torch.manual_seed(0)

In [ ]:
# Print the dimensionality of the data
print('The dimension of the train data:')
print(train_set.data.size())
print('\nThe dimension of the train labels:')
print(train_set.targets.size())

print('\n\nThe dimension of the validation data:')
print(test_set.data.size())
print('\nThe dimension of the validation labels:')
print(test_set.targets.size())

In [ ]:
# Display image resolution and number of classes
input_data_visual, label_visual = next(iter(train_loader))

print("Dimension of input data: {}".format(input_data_visual.size()))
print("Dimension of labels: {}".format(label_visual.size()))

plt.title("Label is: {}".format(label_visual[0]), FontSize=16)
plt.imshow(input_data_visual[0,:,:,:].numpy().reshape(28,28), cmap="gray_r")

In [ ]:
# Check if CUDA GPU training is available
print(torch.cuda.is_available())

In [ ]:
# Set hyperparameters
n_epochs = 50
learning_rate = 0.1

# Set frequency at which loss and accuracy is printed while training
log_interval = 10

# Initialise empty arrays for storing loss and accuracy between epochs
loss_epoch_array = []
train_accuracy = []
test_accuracy = []
valid_accuracy = []

# Train on CUDA GPU if available, otherwise on local CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define CNN class
class CNN_Net(nn.Module):

    # In the constructor below we define the layers of the network
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(5,5))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(5,5))
        self.max_pool1 = nn.MaxPool2d(kernel_size=(2,2))
        self.max_pool2 = nn.MaxPool2d(kernel_size=(2,2))
        self.fc1   = nn.Linear(64*4*4, 1024)
        self.fc2   = nn.Linear(1024, 256)
        self.fc3   = nn.Linear(256, 10)
        self.dropout = nn.Dropout(p=0.3)
    
    # Instantiate the layers of the network, and pass them through activation functions
    def forward(self, x):
        # Activation function can be adjusted here
        x = F.relu(self.conv1(x))
        x = self.max_pool1(x)
        x = F.relu(self.conv2(x))
        x = self.max_pool2(x)
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Define initialisation function for network weights
def weights_init(model):
    if isinstance(model, nn.Linear):
        # Xavier Distribution
        torch.nn.init.xavier_uniform_(model.weight)

# Define train loop
def train(dataloader, epoch):
    # Ensure cumulative epoch loss variable is zeroed at the beginning of every epoch
    loss_epoch = 0
    for i, (data,targets) in enumerate(dataloader):
        # Inside this loop the gradient is zeroed, label predictions are calculated, and then
        # losses are backpropagated through the network and weights are adjusted
        data, targets = data.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = cnn_net(data)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        # Cumulative loss is stored
        loss_epoch += loss.item()
        if i % log_interval == 0:
            # At a certain interval, the current accuracy and loss in a given epoch are printed
            accuracy = float(test(test_loader))
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.4f}, Accuracy: {:.4f}%'.format(
                epoch, i * 32, len(dataloader.dataset), 100.0 * i / len(dataloader), 
                loss.item(),accuracy))
    return loss_epoch

# Define test loop
def test(dataloader):
    # Ensure cumulative epoch loss and correct count variables are zeroed at the begining of every epoch
    test_loss = 0
    correct = 0
    for i, (data, targets) in enumerate(dataloader):
        # Inside this loop, label predictions are calculated and compared with the ground truth 
        # class label. When a correct prediction is made the correct variable is incremented
        data, targets = data.to(device), targets.to(device)
        outputs = cnn_net(data)
        _, pred = torch.max(outputs, 1)
        test_loss += targets.size(0)
        correct += torch.sum(pred == targets)
    return 100.0 * correct / test_loss

# Instantiate CNN class on CUDA/ CPU (depending on which is available)
cnn_net = CNN_Net().to(device)
# Initialise weights to Xavier Uniform using weights_init() function
cnn_net.apply(weights_init)
# Define loss function as Cross Entropy Loss
loss_function = nn.CrossEntropyLoss()
# Define optimizer as Stochastic Gradient Descent with no momentum
optimizer = torch.optim.SGD(cnn_net.parameters(), lr=learning_rate)
# Print structure of model
print(cnn_net)

In [ ]:
# Driver code to iterate through epochs and store loss and accuracy
for epoch in range(n_epochs):
    loss_epoch = 0
    loss_epoch = train(train_loader, epoch)
    loss_epoch_array.append(loss_epoch)
    
    train_accuracy.append(test(train_loader))
    valid_accuracy.append(test(test_loader))
    print("Epoch {}: loss: {:.4f}, train accuracy: {:.4f}, valid accuracy:{:.4f}".format(epoch + 1, 
                                        loss_epoch_array[-1], train_accuracy[-1], valid_accuracy[-1]))

In [ ]:
# Display accuracy results in a figure
plt.rcParams['figure.dpi'] = 300
epochs = range(n_epochs)
plt.plot(epochs, train_accuracy, 'g', label='Training accuracy')
plt.plot(epochs, valid_accuracy, 'b', label='Test accuracy')
plt.title('Training and Test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()